In [13]:
import torch
import torchio as tio

In [19]:
tensor_4d = torch.rand(4, 100, 100, 100)
subject_a = tio.Subject(
    t1=tio.ScalarImage(tensor=tensor_4d),
    label=tio.LabelMap(tensor=(tensor_4d > 0.5)),
    diagnosis='negative',
)

tensor_4d = torch.rand(5, 100, 100, 100)
subject_b = tio.Subject(
    t1=tio.ScalarImage(tensor=tensor_4d),
    label=tio.LabelMap(tensor=(tensor_4d > 0.5)),
    diagnosis='negative',
)

tensor_4d = torch.rand(7, 100, 100, 100)
subject_c = tio.Subject(
    t1=tio.ScalarImage(tensor=tensor_4d),
    label=tio.LabelMap(tensor=(tensor_4d > 0.5)),
    diagnosis='negative',
)

subjects_list = [subject_a, subject_b, subject_c]

In [20]:
rescale = tio.RescaleIntensity(out_min_max=(0, 1))

# As RandomAffine is faster then RandomElasticDeformation, we choose to
# apply RandomAffine 80% of the times and RandomElasticDeformation the rest
# Also, there is a 25% chance that none of them will be applied
spatial = tio.OneOf({
        tio.RandomAffine(): 0.8,
        tio.RandomElasticDeformation(): 0.2,
    },
    p=0.75,
)

# Transforms can be composed as in torchvision.transforms
transforms = [rescale, spatial]
transform = tio.Compose(transforms)


In [21]:
subjects_dataset = tio.SubjectsDataset(subjects_list, transform=transform)

# Images are processed in parallel thanks to a SubjectsLoader
# (which inherits from torch.utils.data.DataLoader)
training_loader = tio.SubjectsLoader(
    subjects_dataset,
    batch_size=4,
    num_workers=4,
    shuffle=True,
)


In [22]:
for subjects_batch in training_loader:
    inputs = subjects_batch['t1'][tio.DATA]
    target = subjects_batch['label'][tio.DATA]
    print('inputs:-',inputs.shape)
    print('target:-',target.shape)

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "C:\Users\Rishabh\anaconda3\envs\3D_Unet_Bleed\Lib\site-packages\torch\utils\data\_utils\worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Rishabh\anaconda3\envs\3D_Unet_Bleed\Lib\site-packages\torch\utils\data\_utils\fetch.py", line 55, in fetch
    return self.collate_fn(data)
           ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Rishabh\anaconda3\envs\3D_Unet_Bleed\Lib\site-packages\torchio\data\loader.py", line 36, in _collate
    collated_value = _stack([subject[key] for subject in subjects])
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Rishabh\anaconda3\envs\3D_Unet_Bleed\Lib\site-packages\torchio\data\loader.py", line 58, in _stack
    collated_dict[key] = _stack([element[key] for element in x])
                         ^^^^^